In [139]:
import pandas as pd
import os
from pathlib import Path

In [140]:
#combining all data to one dataset
current_directory=Path.cwd()
main_folder_path = f"{current_directory}/data/"
#main_folder_path = "/Users/alexandr/Desktop/HomeTask_08/data/"
dataset = pd.DataFrame()
folders = ["idle","running","stairs","walking"]
list_of_items = list()
for name in folders:
    path = f"{main_folder_path}/{name}"
    if os.path.isdir(path):
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isfile(item_path) and Path(item_path).suffix == ".csv":
                try:
                    info = pd.read_csv(item_path)
                    info['activity'] = name
                    list_of_items.append(info)
                except Exception as e:
                    print(f"A mistake occured while readinthe file {item_path}: {e}")
                
dataset = pd.concat(list_of_items, ignore_index=True)
dataset.to_csv("general_dataset.csv")

In [141]:
print(dataset["activity"].value_counts(), "\n")

print("Dataset size is:", dataset.shape, "\n")

print("Dataset columns' type:", "\n", dataset.dtypes)

print(dataset.isna().sum())

dataset.head()

activity
running    102240
walking     55500
idle        31170
stairs       4950
Name: count, dtype: int64 

Dataset size is: (193860, 4) 

Dataset columns' type: 
 accelerometer_X    float64
accelerometer_Y    float64
accelerometer_Z    float64
activity            object
dtype: object
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
activity           0
dtype: int64


,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,0.009577,5.937620,7.570466,idle
1,0.086191,6.555324,7.785944,idle
2,0.004788,6.440403,7.139510,idle
3,0.277727,6.430826,7.690176,idle
4,-0.047884,6.239290,7.340623,idle


In [142]:
#standartizing parameters
columns = dataset.columns.tolist()

for column in columns[0:-1]:
    dataset[column]=(dataset[column]-dataset[column].mean())/dataset[column].std()
dataset.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.227722,0.347865,0.801710,idle
1,-0.218607,0.397384,0.831673,idle
2,-0.228292,0.388171,0.741785,idle
3,-0.195818,0.387403,0.818356,idle
4,-0.234559,0.372048,0.769750,idle


In [143]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#encoding classes with numbers 
le = LabelEncoder()
dataset["activity"] = le.fit_transform(dataset["activity"])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# dividing data into 2 parts: X(parametrs) and y(target)
X = dataset[["accelerometer_X","accelerometer_Y","accelerometer_Z"]]
y = dataset["activity"]

#splitting all data into train and test arrays
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

# To see the mapping of encoded labels to original class names:

print(label_mapping)


{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}


In [144]:
#from sklearn.preprocessing import StandardScaler
#scaler=StandardScaler()
#X_train=scaler.fit_transform(X_train)
#X_test=scaler.fit_transform(X_test)

In [145]:
dataset.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.227722,0.347865,0.801710,0
1,-0.218607,0.397384,0.831673,0
2,-0.228292,0.388171,0.741785,0
3,-0.195818,0.387403,0.818356,0
4,-0.234559,0.372048,0.769750,0


In [146]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

In [147]:
#RandomForestClassifier
model_random_tree = RandomForestClassifier(n_estimators=100)
model_random_tree.fit(X_train, y_train)

y_pred = model_random_tree.predict(X_test)


In [148]:
#calculating of the RandomForestClassifier model metrics 
accuracy=accuracy_score(y_test, y_pred)
conf_matrix= confusion_matrix(y_test, y_pred)
precision=precision_score(y_test, y_pred, average='weighted')
recall=recall_score(y_test, y_pred,average='weighted')
f1=f1_score(y_test, y_pred, average='weighted')

print("RandomForestClassifier Accurasy: ",accuracy,"\n")
print("RandomForestClassifier confusion_matrix:","\n",conf_matrix,"\n")
print("RandomForestClassifier Precision: ", precision,"\n")
print("RandomForestClassifier Recall",recall,"\n")
print("RandomForestClassifier f1: ", f1,"\n")

RandomForestClassifier Accurasy:  0.9997678737233054 

RandomForestClassifier confusion_matrix: 
 [[ 6234     0     0     0]
 [    0 20448     0     0]
 [    0     1   985     4]
 [    0     4     0 11096]] 

RandomForestClassifier Precision:  0.9997679052490089 

RandomForestClassifier Recall 0.9997678737233054 

RandomForestClassifier f1:  0.9997677183664005 



In [149]:
#Classification report
print("Classification Report for RandomForestClassifier:")
print(classification_report(y_test, y_pred))

Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6234
           1       1.00      1.00      1.00     20448
           2       1.00      0.99      1.00       990
           3       1.00      1.00      1.00     11100

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



In [150]:
#SV Classifier
model_svc = SVC(kernel="rbf", C=2, max_iter=20000)
model_svc.fit(X_train, y_train)

y_pred_svc = model_svc.predict(X_test)

/Users/alexandr/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [151]:
#calculating of the SVC model metrics 
accuracy = accuracy_score(y_test, y_pred_svc)
conf_matrix=confusion_matrix(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

print("SV Classifier Accurasy: ",accuracy,"\n")
print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
print("SV Classifier Precision: ", precision,"\n")
print("SV Classifier Recall",recall,"\n")
print("SV Classifier f1: ", f1,"\n")

SV Classifier Accurasy:  0.8950015475085112 

SV Classifier confusion_matrix: 
 [[ 6145    81     0     8]
 [  203 18574     0  1671]
 [    3   277     7   703]
 [   22  1103     0  9975]] 

SV Classifier Precision:  0.9006025432882827 

SV Classifier Recall 0.8950015475085112 

SV Classifier f1:  0.8845327369059 



In [152]:
#classification report of SVC
print("Classification Report for SV Classifier:")
print(classification_report(y_test, y_pred_svc))

Classification Report for V Classifier:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6234
           1       0.93      0.91      0.92     20448
           2       1.00      0.01      0.01       990
           3       0.81      0.90      0.85     11100

    accuracy                           0.90     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.90      0.88     38772



In [153]:
#GridSearch classifier
#from sklearn.model_selection import GridSearchCV

#param_grid = {
#    'C': [0.1,1,3,5],
#    'kernel': ['linear','rbf'],
#    'gamma':['scale','auto'],
#    'class_weight': ['balanced'],
#    'max_iter':[10000]
#}

#grid_svm=GridSearchCV(SVC(), param_grid, refit=True, scoring='precision_weighted', cv=5)
#grid_svm.fit(X_train,y_train)

#y_pred_svc=grid_svm.predict(X_test)

In [154]:
#accuracy = accuracy_score(y_test, y_pred_svc)
#conf_matrix=confusion_matrix(y_test, y_pred_svc)
#precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
#recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
#f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

#print("SV Classifier Accurasy: ",accuracy,"\n")
#print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
#print("SV Classifier Precision: ", precision,"\n")
#print("SV Classifier Recall",recall,"\n")
#print("SV Classifier f1: ", f1,"\n")

In [167]:
# Inferences. Comparing models before extracting Time domain features:
'''
{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}

Основные выводы o Classification Report for V Classifier:
Модель хорошо классифицирует классы 0 и 1, демонстрируя высокие значения precision, recall и F1-score.
Класс 2 является проблемным: хотя точность (precision) 100%, но recall очень низок (1%). 
Это указывает на то, что модель почти не находит объекты этого класса, несмотря на высокую точность в случаях, 
когда они предсказаны.
Класс 3 также демонстрирует приемлемые результаты, хотя precision немного ниже по сравнению с другими классами.
Средние показатели (macro avg) значительно снижены из-за низкой производительности по классу 2, 
и взвешенные метрики отражают общее хорошее качество модели для больших классов, но не для маленьких.


Основные выводы Classification Report for RandomForestClassifier:
Модель RandomForestClassifier показывает отличное результаты для всех классов, с точностью, полнотой и f1-score, 
близкими к 1.00.
Класс 2 показывает гораздо более улучшенные результаты по сравнению с моделью SVC — полнота порядка 0.97, что означает, 
что модель находит почти все объекты этого класса.
Общая точность модели составляет 100%, что указывает на отличную работу модели с текущими данными.

Эта модель показывает лучшее качество классификации, чем предыдущая SVC, особенно для класса 2 and class 3. 

'''


"\n{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}\n\nОсновные выводы o Classification Report for V Classifier:\nМодель хорошо классифицирует классы 0 и 1, демонстрируя высокие значения precision, recall и F1-score.\nКласс 2 является проблемным: хотя точность (precision) 100%, но recall очень низок (1%). \nЭто указывает на то, что модель почти не находит объекты этого класса, несмотря на высокую точность в случаях, \nкогда они предсказаны.\nКласс 3 также демонстрирует приемлемые результаты, хотя precision немного ниже по сравнению с другими классами.\nСредние показатели (macro avg) значительно снижены из-за низкой производительности по классу 2, \nи взвешенные метрики отражают общее хорошее качество модели для больших классов, но не для маленьких.\n\n\nОсновные выводы Classification Report for RandomForestClassifier:\nМодель RandomForestClassifier показывает отличное результаты для всех классов, с точностью, полнотой и f1-score, \nблизкими к

In [156]:
#creating windows and calculating Time domain features using tsfresh 
from tsfresh import extract_features

#selected parametrs for extracting
custom_fc_parameters = {
    "mean": None,
    "variance": None,
    "standard_deviation": None,
    "median": None,
    "maximum": None,
    "minimum": None,
    "absolute_sum_of_changes": None,
    "skewness": None,  
    "kurtosis": None,  
    "quantile": [
        {"q": 0.25},  
        {"q": 0.75}, 
        {"q": 0.50}  
    ],
    "index_mass_quantile": [
        {"q": 0.25}, 
        {"q": 0.75}
    ],
    "root_mean_square": None,
    "range_count": [
        {"max": 1000000000000.0, "min": 0}, 
        {"max": 0, "min": -1000000000000.0}
    ]
}

#window parametrs
window_size = 4
step_size = 2

#list with all time domain features
all_features = list()

#groupping data by activity
for activity_class, group in dataset.groupby("activity"):
    #create windows inside of group
    for i in range(0,len(group)-window_size+1, step_size):
        window = group.iloc[i:i + window_size].copy()  # Извлекаем окно и создаём копию
        window.loc[:, "id_class"] = i // step_size + len(all_features)  # Присваиваем уникальный id_class
        all_features.append(window)

#concatenation all windows in one DataFrame
dataset_with_windows=pd.concat(all_features)

#extracting Time domain features using tsfresh
data_for_tsfresh=dataset_with_windows.drop(columns=["activity"])
extracted_features = extract_features(data_for_tsfresh, column_id="id_class", default_fc_parameters=custom_fc_parameters)

#add back the target value "activity"
final_set = extracted_features.copy()
final_set["activity"] = dataset_with_windows.groupby("id_class")["activity"].first().values

final_set.head()

Feature Extraction: 100%|█████████████████████| 30/30 [00:30<00:00,  1.02s/it]


,accelerometer_X__mean,accelerometer_X__variance,accelerometer_X__standard_deviation,accelerometer_X__median,accelerometer_X__maximum,accelerometer_X__minimum,accelerometer_X__absolute_sum_of_changes,accelerometer_X__skewness,accelerometer_X__kurtosis,accelerometer_X__quantile__q_0.25,...,accelerometer_Z__kurtosis,accelerometer_Z__quantile__q_0.25,accelerometer_Z__quantile__q_0.75,accelerometer_Z__quantile__q_0.5,accelerometer_Z__index_mass_quantile__q_0.25,accelerometer_Z__index_mass_quantile__q_0.75,accelerometer_Z__root_mean_square,accelerometer_Z__range_count__max_1000000000000.0__min_0,accelerometer_Z__range_count__max_0__min_-1000000000000.0,activity
0,-0.217610,0.000173,0.013155,-0.223164,-0.195818,-0.228292,0.051275,1.539513,2.087723,-0.227864,...,2.174469,0.786729,0.821685,0.810033,0.25,1.00,0.799120,4.0,0.0,0
2,-0.225443,0.000320,0.017894,-0.231425,-0.195818,-0.243104,0.079760,1.489692,2.493862,-0.236695,...,2.331555,0.703666,0.781902,0.755768,0.25,0.75,0.734805,4.0,0.0,0
4,-0.227295,0.000269,0.016415,-0.233134,-0.199806,-0.243104,0.051844,1.600646,2.939733,-0.236695,...,3.338073,0.719147,0.778905,0.766088,0.25,1.00,0.736771,4.0,0.0,0
6,-0.208209,0.000859,0.029315,-0.215758,-0.163914,-0.237407,0.141290,0.853102,-1.093821,-0.233134,...,0.398655,0.754103,0.827678,0.784398,0.50,1.00,0.799705,4.0,0.0,0
8,-0.202654,0.000704,0.026542,-0.204648,-0.163914,-0.237407,0.127617,0.360061,0.549085,-0.218179,...,2.292938,0.767586,0.810199,0.781735,0.50,0.75,0.798249,4.0,0.0,0


In [157]:
# selecting only significant fetures

#from tsfresh import select_features

#finalset_clean = final_set.dropna(axis=1)
#relevant_finalset = select_features(finalset_clean.drop(columns=["activity"]), y=finalset_clean["activity"])
#relevant_finalset_with_target = relevant_finalset.copy()
#relevant_finalset_with_target["activity"] = finalset_clean["activity"].values
#relevant_finalset_with_target.head()

In [158]:
#standartizing of parametrs
columns = final_set.columns.tolist()

for column in columns[0:-1]:
    final_set[column]=(final_set[column]-final_set[column].mean())/final_set[column].std()
final_set.head()

,accelerometer_X__mean,accelerometer_X__variance,accelerometer_X__standard_deviation,accelerometer_X__median,accelerometer_X__maximum,accelerometer_X__minimum,accelerometer_X__absolute_sum_of_changes,accelerometer_X__skewness,accelerometer_X__kurtosis,accelerometer_X__quantile__q_0.25,...,accelerometer_Z__kurtosis,accelerometer_Z__quantile__q_0.25,accelerometer_Z__quantile__q_0.75,accelerometer_Z__quantile__q_0.5,accelerometer_Z__index_mass_quantile__q_0.25,accelerometer_Z__index_mass_quantile__q_0.75,accelerometer_Z__root_mean_square,accelerometer_Z__range_count__max_1000000000000.0__min_0,accelerometer_Z__range_count__max_0__min_-1000000000000.0,activity
0,-0.340956,-0.789082,-1.389062,-0.272275,-0.897432,0.869122,-1.281468,1.206265,0.622009,0.319988,...,0.703996,1.870285,0.807663,1.423663,-0.943403,1.028810,-0.236751,1.252828,-1.922092,0
2,-0.353249,-0.788894,-1.378611,-0.286728,-0.897432,0.845521,-1.268825,1.161389,0.773540,0.302601,...,0.763388,1.742233,0.742444,1.332698,-0.943403,-0.405961,-0.384435,1.252828,-1.922092,0
4,-0.356155,-0.788959,-1.381872,-0.289718,-0.900617,0.845521,-1.281216,1.261329,0.939894,0.302601,...,1.143942,1.766099,0.737533,1.349999,-0.943403,1.028810,-0.379919,1.252828,-1.922092,0
6,-0.326203,-0.788206,-1.353420,-0.259318,-0.871949,0.854598,-1.241516,0.587988,-0.565025,0.309612,...,0.032580,1.819988,0.817486,1.380692,0.491313,1.028810,-0.235408,1.252828,-1.922092,0
8,-0.317486,-0.788404,-1.359536,-0.239883,-0.871949,0.854598,-1.247585,0.143888,0.047943,0.339057,...,0.748788,1.840774,0.788834,1.376228,0.491313,-0.405961,-0.238750,1.252828,-1.922092,0


In [159]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
final_set["activity"] = le.fit_transform(final_set["activity"])

X = final_set.drop(columns=["activity"])
y = final_set["activity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [160]:
#RandomForestClassifier
model_random_tree = RandomForestClassifier(n_estimators=100)
model_random_tree.fit(X_train, y_train)

y_pred = model_random_tree.predict(X_test)


In [161]:
accuracy=accuracy_score(y_test, y_pred)
conf_matrix= confusion_matrix(y_test, y_pred)
precision=precision_score(y_test, y_pred, average='weighted')
recall=recall_score(y_test, y_pred,average='weighted')
f1=f1_score(y_test, y_pred, average='weighted')

print("RandomForestClassifier Accurasy: ",accuracy,"\n")
print("RandomForestClassifier confusion_matrix:","\n",conf_matrix,"\n")
print("RandomForestClassifier Precision: ", precision,"\n")
print("RandomForestClassifier Recall",recall,"\n")
print("RandomForestClassifier f1: ", f1,"\n")


RandomForestClassifier Accurasy:  0.9988623435722411 

RandomForestClassifier confusion_matrix: 
 [[3116    1    0    0]
 [   0 8666    0    0]
 [   0    1  479   15]
 [   0    0    3 5299]] 

RandomForestClassifier Precision:  0.9988596940496733 

RandomForestClassifier Recall 0.9988623435722411 

RandomForestClassifier f1:  0.9988557338518641 



In [162]:
print("Classification Report for V Classifier after extracting time domain features:")
print(classification_report(y_test, y_pred))

Classification Report for V Classifier after extracting time domain features:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3117
           1       1.00      1.00      1.00      8666
           2       0.99      0.97      0.98       495
           3       1.00      1.00      1.00      5302

    accuracy                           1.00     17580
   macro avg       1.00      0.99      0.99     17580
weighted avg       1.00      1.00      1.00     17580



In [163]:
#SV Classifier
model_svc = SVC(kernel="rbf", C=2, max_iter=20000)
model_svc.fit(X_train, y_train)

y_pred_svc = model_svc.predict(X_test)

In [164]:
accuracy = accuracy_score(y_test, y_pred_svc)
conf_matrix=confusion_matrix(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

print("SV Classifier Accurasy: ",accuracy,"\n")
print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
print("SV Classifier Precision: ", precision,"\n")
print("SV Classifier Recall",recall,"\n")
print("SV Classifier f1: ", f1,"\n")

SV Classifier Accurasy:  0.9925483503981798 

SV Classifier confusion_matrix: 
 [[3117    0    0    0]
 [   3 8663    0    0]
 [   0    2  371  122]
 [   0    2    2 5298]] 

SV Classifier Precision:  0.9926624163368033 

SV Classifier Recall 0.9925483503981798 

SV Classifier f1:  0.9920841538971918 



In [165]:
print("Classification Report for SV Classifier after extracting time domain features:")
print(classification_report(y_test, y_pred_svc))

Classification Report for V Classifier after extracting time domain features:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3117
           1       1.00      1.00      1.00      8666
           2       0.99      0.75      0.85       495
           3       0.98      1.00      0.99      5302

    accuracy                           0.99     17580
   macro avg       0.99      0.94      0.96     17580
weighted avg       0.99      0.99      0.99     17580



In [166]:
# Inferences. Comparing models after extracting Time domain features:
'''
{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}

Основные выводы Classification Report for SV Classifier:
Очень высокая точность модели для классов 0, 1, и 3, где все метрики практически идеальны.
Для класса два предсказания хуже чем для вышеперечисленных классов, однако они значительно выше чем для этой же модели
Без извлечения временных признаков. По отношению к прошлому результату данной модели без извлечения временных признаков
Когда модель показала recall на уровне 0.01 сейчас модель показывает метрику recall на уровне 0.78, что является существенным
Улучшением работы модели.
Хотя показател recall 0.78 все ещё указывает на то, что модель пропускает некоторые объекты этого класса или путает их с другими классами.
В целом, модель очень эффективна, и ее результаты близки к идеальным, но стоит обратить внимание на класс 2(stairs), 
так как он может быть труднее для классификации. 
Кроме того необходимо отметить, что без извлечения признаков даже при количестве
Итераций 20 000 модель так и не сошлась, о чем была выдана предупреждение. после извлечения признаков данное предупреждение
Отсутствует, что означает что модель нашла оптимальные  веса.


Основные выводы Classification Report for RandomForestClassifier:
Модель достигла практически идеальных результатов после извлечения временных доменных признаков, 
с точностью, полнотой и F1-оценками близкими к 1.0 для всех классов.
Даже классы с меньшей поддержкой (например, класс 2) показали очень высокие метрики.
Извлечение временных доменных признаков дало модели важные дополнительные данные для улучшения классификации.

Сравнение двух моделей:
Можно сказать что классы 0(idle) и 1(running) обе модели предсказывают идеально со стопроцентный точностью. Причём для модели RandomForestClassifier
Извлечение временных признаков не принесло существенного роста в предсказаниях и в метриках, так как они до этого были
На уровне 100%. Для модели SVC извлечения временных признаков дало существенный рост метрик по классам 0 и 1.
Класс 3(walking) модель RandomForestClassifier предсказывает немного лучше чем модель SVC, однако расхождение в предсказаниях
По данному классу не столь критичны.

Наиболее уязвимым местом модели SVC являeтся класс номер 2 (stairs). Анализирую метрике можно сказать что модель RandomForestClassifier
превосходит ee существенно. При предсказании данного класса особенно видна эффективность извлечения временных меток 
и особенно для модели SVC.

Выбирая из двух моделей именно для данных целей(задачи связанные с работой акселерометра) я бы выбрал модель RandomForestClassifier.

'''





"\n{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}\n\nОсновные выводы Classification Report for V Classifier:\nОчень высокая точность модели для классов 0, 1, и 3, где все метрики практически идеальны.\nfor class 2 worse but way more better than without time domain features (0.01 to 0.78)\nЭто указывает на то, что модель пропускает некоторые объекты этого класса или путает их с другими классами.\nВ целом, модель очень эффективна, и ее результаты близки к идеальным, но стоит обратить внимание на класс 2, \nтак как он может быть труднее для классификации.\n\n\nОсновные выводы Classification Report for RandomForestClassifier:\nМодель достигла практически идеальных результатов после извлечения временных доменных признаков, \nс точностью, полнотой и F1-оценками близкими к 1.0 для всех классов.\nДаже классы с меньшей поддержкой (например, класс 2) показали очень высокие метрики.\nИзвлечение временных доменных признаков дало модели важные дополните